In [1]:
DATA_PATH = '../../data/Classification/processed/2_preprocessed_df.pkl'
SIZE=32

In [2]:
import numpy as np
import pandas as pd
import os
from glob import glob
from PIL import Image

In [3]:
raw_df = pd.read_pickle(DATA_PATH)

In [4]:
image_path = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join('data/HAM10000/', '*', '*.jpg'))}

In [5]:
#Define the path and add as a new column
raw_df['path'] = raw_df['image_id'].map(image_path.get)
#Use the path to read images.
raw_df['image'] = raw_df['path'].map(lambda x: np.asarray(Image.open(x).resize((SIZE,SIZE))))

AttributeError: 'NoneType' object has no attribute 'read'